## Taking a look around in R and RStudio

Notice the various windows. 

1. The Console allows you to enter commands and view results
2. The Environment window will show you the various objects R is keeping track of (vectors, datasets, lists, etc.)
3. The bottom right window will show you files, figures/graphs, help files, and more. 
4. If you create/open a script, it will appear in the top-right by default. I encourage using an R project and R script for nearly all work. 

## Start an RStudio project

Steps:

- File &rarr; New Project
- Choose a name for the project and folder location.^[I have a folder called "labs" located in the my main PUBG 511 folder. If you are primarily going to be using RStudio through the computer lab or the Virtual App, then place this folder in your H:\ network space]
- Now start a new script using the sheet with a plus sign icon from the toolbar or using the file menu. On the right side of your window, it should show the name of your project.
- RStudio projects make it so you don't need to worry about setting a working directory - it is defined in the project. Just make sure all files written to or read by R are in the same folder or in a nested subfolder. 

## Installing and loading packages

R is open-source, and, frankly, kinda stinks on its own.^[This is called base R.] But there are many, many user-generated packages that improve R's functionality. We'll be using these packages all the time, especially a group of packages called [the tidyverse](https://www.tidyverse.org/packages/). 

You only need to install the package once and then you're good to go (until it needs updating). But you also need to load the package in every R session if you want to use those commands. 

In [ ]:
# Install the required packages if not already installed 
# By the way, the hashtag/pound/octothorpe symbol will comment out a line in your script

 install.packages(c('tidyverse', 'haven', 'dataverse'))

# let's load your packages in the R session

library(tidyverse)
library(haven)
library(dataverse)

## Some basic built-in functions

R can handle a great diversity of *objects* including lists, variables, names, vectors, data frames, scalars, and plots. Let's create a vector of data using a random draw from a normal distribution and then use two functions to describe the variable.

In [ ]:
x <- rnorm(2500, mean = 50, sd = 10)
mean(x)
summary(x)

We can check the distribution of our variable using a histogram.

In [ ]:
hist(x)

We could change the number of bins used by the histogram if we want and change the y-axis from a frequency to a probability (much better). R options will typically require a value (like the 30 in the breaks option below) or a logical entry of TRUE or FALSE to toggle settings on or off.

In [ ]:
hist(x, breaks = 30, probability = TRUE)

Let's run hist() once more, this time suppressing the data and asking R to give use the output in the console.

In [ ]:
hist(x, breaks = 30, probability = TRUE, plot = FALSE)

## Opening datasets

R has various formats for datasets, typically called a data frame. We could download the ABH_full_district file from [The Journal of Politcs' Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VR12G4) using your web browser. Dataverse repositories allow files to be downloaded in multiple formats: usually as a Stata file, an R workspace, or as a .csv or .tab delimited text file. If you were to place your files in the same folder that you have set as your project location, you would not need to specify working directories or complete file paths; your computer will already know were to look for the data.

We can load an Rdata workspace using the *load()* function. To open the Stata .dta file, we'll need to use functionality from the *read_dta()* function from the **haven** package. But instead, let's use the functionality of the **dataverse** package to directly ingest the data from the online repository in to R. 

In [ ]:
ABH.data <- get_dataframe_by_doi(                    # we'll name the dataframe ABH.data
          filedoi = "doi:10.7910/DVN/VR12G4/WGIHIT", # this is the code for the specific table we want
          original = TRUE,                           # original is Stata 13 binary in this dataset
          .f = haven::read_dta,                      # to read .dta files, calling up haven's read_dta function
          server = "dataverse.harvard.edu")          

## Using the pipe (|>)

The pipe operator allows users to pass objects onto multiple functions without creating new variables or nesting functions. The original pipe is built in to the **tidyverse** packages (%>%) - this is known as the **magrittr** pipe, and you'll see many example codes online using it. It's great. However, starting in R version 4.1, base R includes a new pipe operator that will work throughout everything in R (|>). Let's look at the example below using **tidyverse**'s *summarize* function, which aggregates data in the ways specfied in the command. In this case, we're summarizing two variables (two separate measures of member ideology, coded so that more extreme ideology is higher) in the ABH.data dataset by asking R to calculate their means. 


In [ ]:
ABH.data |> summarize(mean_nom = mean(abs_dist_nom, na.rm = TRUE), 
                      mean_cvp = mean(abs_dist_cvp, na.rm = TRUE))

So here, the pipe passes the data (ABH.data) to the summarize function. We could have specified the data directly inside the summarize function if we wanted to. But using the pipe will let us have a great deal more control over the how we process the data. 

## **filter** and **group_by**

These functions are also part of the **tidyverse**. *filter()* allows us to, well, filter our observations based on some condition or a list of conditions. In the code below, we select just those districts that come after the 105th Congress. 

The *group_by()* function allows you to aggregate your data by grouping variables. In the code below, we group the data by congress and whether the member is in their first term or not (coded 1 or 0) prior to calculating the means. 

In [ ]:
ABH.data |> filter(congress>105) |> 
       group_by(congress, first_term) |> 
                summarize(mean_nom = mean(abs_dist_nom, na.rm = TRUE), 
                          mean_cvp = mean(abs_dist_cvp, na.rm = TRUE))

The pipe first passes the data to be filtered, then passes the filtered dataset to be grouped, then passes the grouped and final data to be summarized. If you want to insert the pipe using a keyboard shortcut in RStudio, you can use <kbd>ctrl</kbd> + <kbd>shift</kbd> + <kbd>m</kbd>. Note that the shortcut uses the **tidyverse** pipe by default, but you can change it to the base R |> pipe in the settings. Check the "Use native pipe operator, |> (requires $ 4.1+) box in Tools &rarr; Global Options &rarr; Code menu window.